In [ ]:
docker ps --filter="name=sql"

In [ ]:
if (!$OriginalProjectPath) {$OriginalProjectPath = (Get-Item .).FullName}
$WorkingDirectory = $OriginalProjectPath + '\WorkingDirectory\LocalDockerExample'
"OriginalProjectPath: $OriginalProjectPath"
"WorkingDirectory: $WorkingDirectory"

In [ ]:
cd $env:TEMP\tf
$TerraformExePath = (Get-ChildItem)[0].FullName

In [ ]:
.\terraform version

In [ ]:
$command = 'docker inspect --format="{{.ID}}" sql'
Write-Output "Executing the command:"
Write-Output $command
$DockerId = Invoke-Expression $command
Write-Output 'Docker Id in question:' $DockerId

In [ ]:
$command = ".\terraform -chdir=$WorkingDirectory init"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command

In [ ]:

$command = ".\terraform -chdir=$WorkingDirectory import docker_container.arbitrarylabel $DockerId"
Write-Output "Executing the command:"
Write-Output $command
Invoke-Expression $command